In [42]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Embedding
from keras.layers import Convolution1D, MaxPooling1D
from keras import backend as K
from keras.wrappers.scikit_learn import KerasClassifier
 
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score
 
import pandas as pd
import MeCab
import numpy as np
 
# 適当なデータを用意してください
df = pd.read_csv('data/book.csv', header=None)
 
#読み込んだcsvから長さ読み込んでもろもろの処理をするメソッドを書く
def init_train():
    print('Loading data')
    Xs = []
    Ys = []
    voc = {}
    maxlen = 0
    maxwords = 0
    buff = set()
    TARGET_DIR = 'data/txt/*'
    idx_name = {}
    for i,filename in enumerate(df[3]):
        idx_name[i] = filename
    for line in df[2]:
        a - list(line)
    #行の最大文字数
        maxlen = max(maxlen, len(a))
        [buff.add(w) for w in a]
    #行数
    maxwords = len(buff)
    #限界にMAX,それぞれの項目に数値を入れている
    voc[maxwords] = '___MAX___'
    voc['___META_MAXWORD___'] = maxwords
    voc['___META_MAXLEN___'] = maxlen
    print("maxwords %d"%maxwords)
    print("maxlen %d"%maxlen)
    print("idx name len %d"%len(idx_name))
    for i,filename in enumerate(df[3]):
        for line in set(filter(lambda x:x!='',df[2])): 
            X = [maxwords]*maxlen
            #余分な空白を消す
            line = line.strip()
            for idx, ch in enumerate(list(line)):
            #辞書vocのch番目が殻だったらvocの長さを代入
                if voc.get(ch) == None:
                    voc[ch] = len(voc)
                convert = voc[ch]
                X[idx] = convert
                #Xs,Ysそれぞれにvocの長さ,倍率を追加している
        Xs.append(X)
        y = [0.]*len(idx_name)
        y[i] = 1.
        Ys.append(y)
    X_train, X_test, y_train, y_test = train_test_split( Xs, Ys, test_size=0.1, random_state=42)
    open('vod.pkl', 'wb').write(pickle.dumps(voc))
    open('idx_name.pkl', 'wb').write(pickle.dumps(idx_name))
    sequence_length = maxlen
    vocabulary_size = maxwords
    embedding_dim   = 256*1
    filter_sizes    = [3,4,5,1,2]
    num_filters     = 512*1
    drop            = 0.5
    
    nb_epoch   = 10
    batch_size = 30
    return sequence_length, embedding_dim, filter_sizes, vocabulary_size, num_filters, drop, idx_name, \
    X_train, X_test, y_train, y_test, batch_size, nb_epoch, Xs, Ys
    
# テキストを分かち書きして返す
def tokenize(text):
    wakati = MeCab.Tagger('-O wakati')
    text = str(text).strip()
    #print(text)
    #encoded_text = text.encode('utf-8')
    return wakati.parse(text)
 
tokenized_text_list = [tokenize(texts) for texts in df[2]]
 
max_features = 5000
maxlen = 600
batch_size = 32
embedding_dim = 50
filter_length = 3
hidden_dims = 250
nb_epoch = 5
 
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokenized_text_list)
seq = tokenizer.texts_to_sequences(tokenized_text_list)
X = sequence.pad_sequences(seq, maxlen=maxlen)
Y = str(df[2].sample(n=10)).strip()

#Y = str(df[2]).strip()
 
# モデルを返すメソッド
def build_model():
    print('Build model...')
    model = Sequential()
 
    # we start off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    model.add(Embedding(max_features,
                        embedding_dims,
                        input_length=maxlen,
                        dropout=0.2))
 
    # we add a Convolution1D, which will learn nb_filter
    # word group filters of size filter_length:
    model.add(Convolution1D(nb_filter=nb_filter,
                            filter_length=filter_length,
                            border_mode='valid',
                            activation='relu',
                            subsample_length=1))
    # we use max pooling:
    model.add(MaxPooling1D(pool_length=model.output_shape[1]))
 
    # We flatten the output of the conv layer,
    # so that we can add a vanilla dense layer:
    model.add(Flatten())
 
    # We add a vanilla hidden layer:
    model.add(Dense(hidden_dims))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
 
    # We project onto a single unit output layer, and squash it with a sigmoid:
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
 
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model
 
model = KerasClassifier(build_fn=build_model, nb_epoch=nb_epoch, batch_size=batch_size)
# evaluate using 10-fold cross validation
print(Y)
kfold = StratifiedKFold(y=Y, n_folds=10, shuffle=True)
results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

2243    明るさは滅びの姿であろうか。人も星も、暗いうちはまだ滅亡せぬ。\r\n――円城塔氏 推薦！\...
3004    「創造的破壊のドキュメンタリー。\r\n経済学に関心がない人にこそ読んでほしい、\r\n最上...
1853    雪に覆われ下界と遮断されたシタフォード村の山荘。そこに集まった隣人たちが退屈しのぎに降霊会を...
1428    〈ツイスト博士シリーズ〉内側から錠がかかった書斎で、ミステリ作家が煮えたぎる鍋に顔と両手を突...
741     進化論から量子論、宇宙論まで20世紀科学の成果は広大だ。ガリレオの打ち立てた基幹から芽生え、...
77      古き良きエドワード朝時代の面影を今なお残すバートラム・ホテル。ミス・マープルも淡い過去の思い...
561     ミステリ史を語るうえで不可欠の名探偵たちが登場する作品を結集した名探偵アンソロジーの決定版第...
1035    日本ＳＦ作家クラブ創立50周年記念特集\r\n1963年3月に創立され、2013年に50周年...
1549    \n大変ご好評によりハヤカワ・オンラインでの特典ポスターが先着購入枚数に達しましたため、10...
1975    凄腕の殺し屋マイケルは、ガールフレンドのエレナの妊娠を機に、組織を抜けようと誓った。育ての親...
Name: 2, dtype: object


IndexError: tuple index out of range